In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ======================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_transformers4rec_getting-started-session-based-01-etl-with-nvtabular/nvidia_logo.png" style="width: 90px; float: right;">

# ETL with NVTabular

In this notebook we are going to generate synthetic data and then create sequential features with [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular). Such data will be used in the next notebook to train a session-based recommendation model.

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems. It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Import required libraries

In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob

# import cudf
import numpy as np
import pandas as pd

import nvtabular as nvt
from nvtabular.ops import *
from merlin.schema.tags import Tags

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")


### Define Input/Output Path

In [2]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "./data/")

In [7]:
INPUT_DATA_DIR

'./data/'

## Create a Synthetic Input Data

In [3]:
NUM_ROWS = os.environ.get("NUM_ROWS", 100000)

In [4]:
long_tailed_item_distribution = np.clip(np.random.lognormal(3., 1., int(NUM_ROWS)).astype(np.int32), 1, 50000)
# generate random item interaction features 
df = pd.DataFrame(np.random.randint(70000, 90000, int(NUM_ROWS)), columns=['session_id'])
df['item_id'] = long_tailed_item_distribution

# generate category mapping for each item-id
df['category'] = pd.cut(df['item_id'], bins=334, labels=np.arange(1, 335)).astype(np.int32)
df['age_days'] = np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)
df['weekday_sin']= np.random.uniform(0, 1, int(NUM_ROWS)).astype(np.float32)

# generate day mapping for each session 
map_day = dict(zip(df.session_id.unique(), np.random.randint(1, 10, size=(df.session_id.nunique()))))
df['day'] =  df.session_id.map(map_day)

Visualize couple of rows of the synthetic dataset:

In [5]:
df.head()

,session_id,item_id,category,age_days,weekday_sin,day
0,78814,17,5,0.935278,0.764605,5
1,87422,13,4,0.527721,0.237671,3
2,87958,16,5,0.387662,0.086010,9
3,70729,47,13,0.956692,0.828015,6
4,72325,26,7,0.444470,0.214215,4


## Feature Engineering with NVTabular

Deep Learning models require dense input features. Categorical features are sparse, and need to be represented by dense embeddings in the model. To allow for that, categorical features first need to be encoded as contiguous integers `(0, ..., |C|)`, where `|C|` is the feature cardinality (number of unique values), so that their embeddings can be efficiently stored in embedding layers.  We will use NVTabular to preprocess the categorical features, so that all categorical columns are encoded as contiguous integers. Note that the `Categorify` op encodes `nulls` to `1`, OOVs to `2` automatically. We preserve `0` for padding. The encoding of other categories starts from `3`. In our synthetic dataset we do not have any nulls. On the other hand `0` is used for padding the sequences in input block. 

Here our goal is to create sequential features. To do so, we are grouping the features together at the session level in the following cell. In this synthetically generated example dataset, we do not have a timestamp column, but if we had one (that's the case for most real-world datasets), we would be sorting the interactions by the timestamp column as in this [example notebook](../end-to-end-session-based/01-ETL-with-NVTabular.ipynb). Note that we also trim each feature sequence in a  session to a certain length. Here, we use the NVTabular library so that we can easily preprocess and create features on GPU with a few lines.

In [6]:
SESSIONS_MAX_LENGTH =20

# Categorify categorical features
categ_feats = ['item_id', 'category'] >> nvt.ops.Categorify()

# Define Groupby Workflow
groupby_feats = categ_feats + ['session_id', 'day', 'age_days', 'weekday_sin']

# Group interaction features by session
groupby_features = groupby_feats >> nvt.ops.Groupby(
    groupby_cols=["session_id"], 
    aggs={
        "item_id": ["list", "count"],
        "category": ["list"],     
        "day": ["first"],
        "age_days": ["list"],
        'weekday_sin': ["list"],
        },
    name_sep="-")

# Select and truncate the sequential features
sequence_features_truncated = (
    groupby_features['category-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
)

sequence_features_truncated_item = (
    groupby_features['item_id-list']
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> TagAsItemID()
)  
sequence_features_truncated_cont = (
    groupby_features['age_days-list', 'weekday_sin-list'] 
    >> nvt.ops.ListSlice(-SESSIONS_MAX_LENGTH) 
    >> nvt.ops.AddMetadata(tags=[Tags.CONTINUOUS])
)

# Filter out sessions with length 1 (not valid for next-item prediction training and evaluation)
MINIMUM_SESSION_LENGTH = 2
selected_features = (
    groupby_features['item_id-count', 'day-first', 'session_id'] + 
    sequence_features_truncated_item +
    sequence_features_truncated + 
    sequence_features_truncated_cont
)
    
filtered_sessions = selected_features >> nvt.ops.Filter(f=lambda df: df["item_id-count"] >= MINIMUM_SESSION_LENGTH)

seq_feats_list = filtered_sessions['item_id-list', 'category-list', 'age_days-list', 'weekday_sin-list'] >>  nvt.ops.ValueCount()

workflow = nvt.Workflow(filtered_sessions['session_id', 'day-first'] + seq_feats_list)

dataset = nvt.Dataset(df)

# Generate statistics for the features and export parquet files
# this step will generate the schema file
workflow.fit_transform(dataset).to_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt"))

/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/opt/intel/oneapi/intelpython/latest/envs/transformer4rec/lib/python3.9/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


It is possible to save the preprocessing workflow. That is useful to apply the same preprocessing to other data (with the same schema) and also to deploy the session-based recommendation pipeline to Triton Inference Server.

In [8]:
workflow.output_schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.domain.min,properties.domain.max,properties.domain.name,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.value_count.min,properties.value_count.max
0,session_id,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,day-first,(),"DType(name='int64', element_type=<ElementType....",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,item_id-list,"(Tags.CATEGORICAL, Tags.ID, Tags.LIST, Tags.ITEM)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.item_id.parquet,0.0,506.0,item_id,507.0,52.0,2.0,14.0
3,category-list,"(Tags.CATEGORICAL, Tags.LIST)","DType(name='int64', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.category.parquet,0.0,185.0,category,186.0,30.0,2.0,14.0
4,age_days-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,14.0
5,weekday_sin-list,"(Tags.CONTINUOUS, Tags.LIST)","DType(name='float32', element_type=<ElementTyp...",True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,14.0


Save NVTabular workflow.

In [9]:
workflow.save(os.path.join(INPUT_DATA_DIR, "workflow_etl"))

## Export pre-processed data by day

In this example we are going to split the preprocessed parquet files by days, to allow for temporal training and evaluation. There will be a folder for each day and three parquet files within each day folder: `train.parquet`, `validation.parquet` and `test.parquet`.

In [10]:
OUTPUT_DIR = os.environ.get("OUTPUT_DIR",os.path.join(INPUT_DATA_DIR, "sessions_by_day"))

In [11]:
# Read in the processed parquet file
sessions_gdf = pd.read_parquet(os.path.join(INPUT_DATA_DIR, "processed_nvt/part_0.parquet"))

In [12]:
sessions_gdf

,session_id,day-first,item_id-list,category-list,age_days-list,weekday_sin-list
0,70000,8,"[103, 25, 60]","[33, 8, 20]","[0.7048097848892212, 0.6903286576271057, 0.667...","[0.2934705317020416, 0.3389386236667633, 0.052..."
1,70001,5,"[27, 4, 5, 15, 17]","[8, 3, 3, 6, 6]","[0.5091915726661682, 0.09631510078907013, 0.16...","[0.01199796050786972, 0.4487989842891693, 0.95..."
2,70002,7,"[7, 11, 18, 18, 14, 44]","[5, 4, 6, 6, 4, 13]","[0.7389612793922424, 0.3185826539993286, 0.915...","[0.4517076313495636, 0.49958136677742004, 0.89..."
3,70003,1,"[6, 119, 55, 91, 12, 11, 7]","[3, 37, 17, 26, 4, 4, 5]","[0.8941054344177246, 0.6713675260543823, 0.135...","[0.32163500785827637, 0.5422960519790649, 0.85..."
4,70004,6,"[55, 57]","[17, 20]","[0.6114782691001892, 0.7743163704872131]","[0.5357045531272888, 0.24191606044769287]"
...,...,...,...,...,...,...
19209,89995,6,"[28, 4, 28, 81, 3, 296]","[11, 3, 11, 23, 3, 72]","[0.7987788319587708, 0.17112861573696136, 0.13...","[0.08514165133237839, 0.1657770872116089, 0.47..."
19210,89996,5,"[14, 14, 42, 35]","[4, 4, 14, 7]","[0.805230438709259, 0.066165991127491, 0.50335...","[0.3799225687980652, 0.5059230327606201, 0.182..."
19211,89997,7,"[5, 32, 7, 134, 16]","[3, 10, 5, 36, 7]","[0.6572467684745789, 0.3500416874885559, 0.884...","[0.914212167263031, 0.04325936362147331, 0.450..."
19212,89998,9,"[6, 6, 23, 87]","[3, 3, 9, 25]","[0.42906442284584045, 0.6422862410545349, 0.33...","[0.6524897813796997, 0.17047034204006195, 0.60..."


In [14]:
from transformers4rec.utils.data_utils import save_time_based_splits
save_time_based_splits(data=nvt.Dataset(sessions_gdf),
                       output_dir= OUTPUT_DIR,
                       partition_col='day-first',
                       timestamp_col='session_id', 
                       cpu=True
                      )

TypeError: 'int' object is not subscriptable

## Check out the preprocessed outputs

In [14]:
TRAIN_PATHS = os.path.join(OUTPUT_DIR, "1", "train.parquet")

In [15]:
df = pd.read_parquet(TRAIN_PATHS)
df.head()

,session_id,item_id-list,category-list,age_days-list,weekday_sin-list
0,70000,"[306, 5, 40, 17]","[104, 3, 12, 6]","[0.044022594, 0.34956282, 0.7326993, 0.09403495]","[0.7417527, 0.60325843, 0.07417604, 0.28911334]"
1,70001,"[43, 20, 69, 8, 57]","[13, 6, 21, 3, 16]","[0.8072543, 0.28916782, 0.04966254, 0.08417622...","[0.7995051, 0.86722755, 0.84298295, 0.15793765..."
2,70002,"[137, 35, 37, 85, 65, 5]","[37, 10, 11, 22, 18, 3]","[0.04696693, 0.94499177, 0.2922437, 0.83047426...","[0.72519076, 0.92308444, 0.40120387, 0.3821016..."
4,70007,"[28, 9, 153, 74, 53, 15, 173]","[9, 4, 39, 20, 15, 5, 46]","[0.4730765, 0.69885534, 0.034774363, 0.7225920...","[0.33613566, 0.660022, 0.72897774, 0.66087157,..."
5,70021,"[59, 32, 11, 21, 23, 23, 9, 15]","[17, 10, 7, 7, 8, 8, 4, 5]","[0.07898139, 0.27463168, 0.1885847, 0.5203435,...","[0.39734098, 0.74895114, 0.43540764, 0.8372503..."


In [16]:
import gc
del df
gc.collect()

512

You have  just created session-level features to train a session-based recommendation model using NVTabular. Now you can move to the the next notebook,`02-session-based-XLNet-with-PyT.ipynb` to train a session-based recommendation model using [XLNet](https://arxiv.org/abs/1906.08237), one of the state-of-the-art NLP model. Please shut down this kernel to free the GPU memory before you start the next one.